ADZUNA Kaggle Project

- Review the nltk NLP library in Python
- Tokenize sentences
- Remove stopwords + Others

NLP in Python: http://www.nltk.org/book/ch01.html

Stem words: http://www.nltk.org/book/ch03.html  (how to remove plural endings and stemming)

Most popular phrases: http://www.markhneedham.com/blog/2015/01/19/pythonnltk-finding-the-most-common-phrases-in-how-i-met-your-mother/

Useful Python Tools: https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/


In [68]:
import pandas as pd                  # import necessary libraries
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from nltk import util
from nltk import ngrams

In [69]:
#Import each of the csv files you downloaded from the SSCC into a pandas df
adzuna = pd.read_csv('~/Dropbox/NU/ADVANCED_MODELING/ADZUNA/datasets/Train_rev1.csv') 

In [70]:
adzuna.head(2)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk


Next we need to loop through each row in the adzuna data frame so that we add (initially) the most common word in a new column of the adzuna data frame. Let's first create a copy of the data frame and remove the unnecessary columns.

In [71]:
collist = ['Id', 'Title', 'FullDescription', 'LocationRaw','LocationNormalized','ContractType','ContractTime',
           'Company','Category','SalaryNormalized','SourceName']  # list columns to include in adzuna dataset.
adzuna = adzuna[collist]

In [72]:
#df1_test = adzuna.head(5000)       # Subset of actual data set for testing purposes
df1_test = adzuna                  # Copy of adzuna data set

In [73]:
def to_lower(des):
    return(des.lower())

In [74]:
df1_test['FullDescription']    = df1_test['FullDescription'].apply(to_lower) 
df1_test['fulldescription']    = df1_test['FullDescription'].apply(to_lower) 
df1_test['locationraw']        = df1_test['LocationRaw'].apply(to_lower) 
df1_test['locationnormalized'] = df1_test['LocationNormalized'].apply(to_lower)

In [75]:
stops = set(stopwords.words("english"))                                     # create set of stop words
stops.update([',','****k','.','****','salary','/','(',')','&','?',';',':'])     # add additional words to stop words 'set'
stops.update(set(df1_test['locationraw']))
stops.update(set(df1_test['locationnormalized']))

In [76]:
def remove_stops(des):
    token = nltk.word_tokenize(des)                                 # split sentence into words                   
    filtered_words = [word for word in token if word not in stops]
    return(filtered_words)

In [77]:
# *****************                 THIS STEP CAN TAKE 10 MINUTES TO RUN                  *****************
df1_test['fulldescription'] = df1_test['fulldescription'].apply(remove_stops)

In [78]:
def get_most_common_word(des):
    filtered_words_freq = nltk.FreqDist(des)                            # get frequency of each word in des text
    return(filtered_words_freq.most_common(1))

In [79]:
df1_test['unigram'] = df1_test['fulldescription'].apply(get_most_common_word)

In [80]:
def extract_keyword(col):
    return(col[0][0])

In [81]:
df1_test['unigram']   = df1_test['unigram'].apply(extract_keyword)

In [82]:
df1_test.head(2)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryNormalized,SourceName,fulldescription,locationraw,locationnormalized,unigram
0,12612628,Engineering Systems Analyst,engineering systems analyst dorking surrey sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,25000,cv-library.co.uk,"[engineering, systems, analyst, client, locate...","dorking, surrey, surrey",dorking,systems
1,12612830,Stress Engineer Glasgow,stress engineer glasgow salary **** to **** we...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,"[stress, engineer, currently, looking, talente...","glasgow, scotland, scotland",glasgow,analysis


In [83]:
df1_test.shape

(244768, 15)

In [84]:
def bigrams(sentence_split):
    twograms = ngrams(sentence_split, 2)
    ngram_list = []
    for gram in twograms:
        ngram_list.append(gram)
        
    desFreq = nltk.FreqDist(ngram_list)
    desFreq = desFreq.most_common(1) 
   
    return(desFreq)

In [85]:
df1_test['bigram'] = df1_test['fulldescription'].apply(bigrams)

In [86]:
df1_test['bigram']   = df1_test['bigram'].apply(extract_keyword)

In [87]:
def trigrams(sentence_split):
    twograms = ngrams(sentence_split, 3)
    ngram_list = []
    for gram in twograms:
        ngram_list.append(gram)
        
    desFreq = nltk.FreqDist(ngram_list)
    desFreq = desFreq.most_common(1) 
   
    return(desFreq)

In [88]:
df1_test['trigram'] = df1_test['fulldescription'].apply(trigrams)

In [89]:
df1_test['trigram']   = df1_test['trigram'].apply(extract_keyword)

In [90]:
df1_test.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryNormalized,SourceName,fulldescription,locationraw,locationnormalized,unigram,bigram,trigram
0,12612628,Engineering Systems Analyst,engineering systems analyst dorking surrey sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,25000,cv-library.co.uk,"[engineering, systems, analyst, client, locate...","dorking, surrey, surrey",dorking,systems,"(systems, analyst)","(engineering, systems, analyst)"
1,12612830,Stress Engineer Glasgow,stress engineer glasgow salary **** to **** we...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,"[stress, engineer, currently, looking, talente...","glasgow, scotland, scotland",glasgow,analysis,"(stress, engineer)","(demonstrate, experience, least)"
2,12612844,Modelling and simulation analyst,mathematical modeller / simulation analyst / o...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,"[mathematical, modeller, simulation, analyst, ...","hampshire, south east, south east",hampshire,analyst,"(successful, candidate)","(analyst, aae, pension)"
3,12613049,Engineering Systems Analyst / Mathematical Mod...,engineering systems analyst / mathematical mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,27500,cv-library.co.uk,"[engineering, systems, analyst, mathematical, ...","surrey, south east, south east",surrey,systems,"(systems, analyst)","(engineering, systems, analyst)"
4,12613647,"Pioneer, Miser Engineering Systems Analyst","pioneer, miser engineering systems analyst do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000,cv-library.co.uk,"[pioneer, miser, engineering, systems, analyst...","surrey, south east, south east",surrey,analyst,"(engineering, systems)","(engineering, systems, analyst)"


In [91]:
collistFinal = ['Id', 'Title', 'locationnormalized','ContractType','ContractTime','Company','Category',
                'SalaryNormalized','SourceName','unigram','bigram','trigram']  
df1_test = df1_test[collistFinal]

In [92]:
df1_test.to_csv('adzuna.csv')